In [4]:
cursor.close()
mydb.close()

In [1]:
#this creates the new database, and only needs to be done the first time

import mysql.connector

from mysql.connector import (connection)

mydb = connection.MySQLConnection(user='root', password='password',
                                 host='127.0.0.1')

print(mydb)

mycursor = mydb.cursor(buffered='True')

mycursor.execute("SHOW DATABASES")

for x in mycursor:
    print(x)

('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('VELA_GEN3',)
('VELA_GEN6',)


In [2]:
import glob
import mysql.connector

from mysql.connector import (connection)

#now we create the table in the database we want to add

mydb = connection.MySQLConnection(user='root', password='password',
                                 host='127.0.0.1', database='VELA_GEN3')

#print(mydb)

cursor = mydb.cursor()

cursor.execute("CREATE TABLE propermvircalc (VELA_number CHAR(2), scale_factor float(53),  rockstar_id INT(255), Rockstar_Mvir float(53), Rockstar_Rvir float(53), New_Rvir_Calculation float(53), New_Rvir_error float(53), Stellar_Mass_1Rvir float(53), Gas_Mass_1Rvir float(53), Darkmatter_Mass_1Rvir float(53), Stellar_Mass_010Rvir float(53), Gas_Mass_010Rvir float(53), Stellar_Mass_015Rvir float(53), Gas_Mass_015Rvir float(53), Stellar_Mass_020Rvir float(53), Gas_Mass_020Rvir float(53), Stellar_Halfmass_Radius_010Rvir float(53), Stellar_Halfmass_Radius_010Rvir_error float(53), Stellar_Halfmass_Radius_015Rvir float(53), Stellar_Halfmass_Radius_015Rvir_error float(53), Stellar_Halfmass_Radius_020Rvir float(53), Stellar_Halfmass_Radius_020Rvir_error float(53))")


#now that the table is created, we read the data we want from each VELA sim file and load it into the table we just created

VELA_numbers = ['06', '07', '08', '09', '10', '11', '12', '13', '14', '15']

for VELA_number in VELA_numbers:
    print(VELA_number)
    properfiles = glob.glob('/Users/user1/documents/GEN3/rockstar_moresnaps/propermvircalc/VELA{}/improvedrvir[0-9][0-9][0-9].ascii'.format(VELA_number))

    values_to_add = []

    for properfile in properfiles:

        readfile = open(properfile)
        lines = readfile.readlines()

        count = 0
        current_scale = float(properfile[-9:-6])/1000
        for line in lines:
            #we want to skip the first line as it is just the headers
            if count == 0:
                count +=1
                continue

            current_line = line.split()
            current_line_sql = [VELA_number, str(current_scale)] + [str(x) for x in current_line]
            values_to_add.append(tuple(current_line_sql))


        readfile.close()
        
    #now that we have the data we want to add, we add it to the Table
    
    sql_add_command = 'INSERT INTO propermvircalc (VELA_number, scale_factor, rockstar_id, Rockstar_Mvir, Rockstar_Rvir, New_Rvir_Calculation, New_Rvir_error, Stellar_Mass_1Rvir, Gas_Mass_1Rvir, Darkmatter_Mass_1Rvir, Stellar_Mass_010Rvir, Gas_Mass_010Rvir, Stellar_Mass_015Rvir, Gas_Mass_015Rvir, Stellar_Mass_020Rvir, Gas_Mass_020Rvir, Stellar_Halfmass_Radius_010Rvir, Stellar_Halfmass_Radius_010Rvir_error, Stellar_Halfmass_Radius_015Rvir, Stellar_Halfmass_Radius_015Rvir_error, Stellar_Halfmass_Radius_020Rvir, Stellar_Halfmass_Radius_020Rvir_error) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
    cursor.executemany(sql_add_command, values_to_add)

    mydb.commit()

    print(cursor.rowcount, "was inserted.")
    
    #now we check that all the data matches and that no errors were made that need to be corrected
    
    
    #NEED TO CHANGE THIS TO ONLY GET ONES FROM VELA_Number
    cursor.execute("SELECT * FROM propermvircalc HAVING VELA_number = {}".format(VELA_number))

    myresult = cursor.fetchall()

    for x, y in zip(myresult, values_to_add):
        
        for x_sub, y_sub in zip(x,y):
            #this is for the rockstar_id, which are stored as a float with number.0 instead of number as they
            #are in the sql database, so to compare I have to make the ones from the ascii files ints
            if x_sub == x[2] and x_sub != int(float(y_sub)):
                print(x_sub, y_sub)
            elif x_sub != x[2] and str(x_sub) != y_sub:
                print(x_sub, y_sub)
            
    print(VELA_number)
cursor.close()
mydb.close()


06
826 was inserted.
06
07
844 was inserted.
07
08
994 was inserted.
08
09
87 was inserted.
09
10
637 was inserted.
10
11
580 was inserted.
11
12
476 was inserted.
12
13
459 was inserted.
13
14
343 was inserted.
14
15
189 was inserted.
15


In [13]:
print(x_sub, y_sub)
print(type(x_sub), type(y_sub))

print(x[2], y[2])

0.005817066049706998 0.005817066049706998
<class 'float'> <class 'str'>
4179040 4179040.0


In [72]:
mycursor.execute("SELECT `COLUMN_NAME` FROM `INFORMATION_SCHEMA`.`COLUMNS` WHERE `TABLE_SCHEMA`='VELA_GEN3' AND `TABLE_NAME`='propermvircalc';")

titles = []
for x in mycursor:
    titles.append(x[0])
print(len(tuple(titles)))



sql_add_command = 'INSERT INTO propermvircalc (VELA_number, rockstar_id, Rockstar_Mvir, Rockstar_Rvir, New_Rvir_Calculation, New_Rvir_error, Stellar_Mass_1Rvir, Gas_Mass_1Rvir, Darkmatter_Mass_1Rvir, Stellar_Mass_010Rvir, Gas_Mass_010Rvir, Stellar_Mass_015Rvir, Gas_Mass_015Rvir, Stellar_Mass_020Rvir, Gas_Mass_020Rvir, Stellar_Halfmass_Radius_010Rvir, Stellar_Halfmass_Radius_010Rvir_error, Stellar_Halfmass_Radius_015Rvir, Stellar_Halfmass_Radius_015Rvir_error, Stellar_Halfmass_Radius_020Rvir, Stellar_Halfmass_Radius_020Rvir_error) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
mycursor.executemany(sql_add_command, values_to_add)

mydb1.commit()

print(mycursor.rowcount, "was inserted.")

20
844 was inserted.


In [83]:
mycursor.execute("SELECT * FROM propermvircalc")

myresult = mycursor.fetchall()

for x, y in zip(myresult, values_to_add):
    for x_sub, y_sub in zip(x,y):
        if x_sub == int(float(x[0])) and str(x_sub) != y_sub:
            print(x_sub, y_sub)
        elif x_sub != x[0] and str(x_sub) != y_sub:
            print(x_sub, y_sub)
            

581484 07
16828571715.1603 581484.0
11.46821651745211 16828571715.1603
12.251456741038426 11.46821651745211
0.0013163701236749148 12.251456741038426
262672564.91698104 0.0013163701236749148
1969204215.4767535 262672564.91698104
18280588176.089523 1969204215.4767535
107088391.15260203 18280588176.089523
85428929.07826541 107088391.15260203
127970524.14944322 85428929.07826541
142200086.43064845 127970524.14944322
232180170.57518706 142200086.43064845
0.546293601324911 232180170.57518706
0.0013163701236744707 0.546293601324911
0.6489704709715207 0.0013163701236744707
0.0013163701236744707 0.6489704709715207
0.7147889771552449 0.0013163701236744707
0.0013163701236744707 0.7147889771552449
619344 07
19914286053.42399 619344.0
12.359287021488582 19914286053.42399
13.064356540506008 12.359287021488582
0.00141865094369642 13.064356540506008
321618879.6705799 0.00141865094369642
2255651405.377352 321618879.6705799
20940273798.517582 2255651405.377352
120586618.26247792 20940273798.517582
10913

506733405.98909545 846940711.2634633
948962859.7789094 506733405.98909545
1006365394.250534 948962859.7789094
982739321.4734789 1006365394.250534
0.5747452797764907 982739321.4734789
0.0018968491081732264 0.5747452797764907
0.6733814334014988 0.0018968491081732264
0.0018968491081732264 0.6733814334014988
0.7340806048630426 0.0018968491081732264
0.0018968491081733374 0.7340806048630426
1273380 07
22728571815.636703 1273380.0
16.407652138942517 22728571815.636703
17.275871564568373 16.407652138942517
0.0018833393180628377 17.275871564568373
249799085.06405243 0.0018833393180628377
2102068086.6951163 249799085.06405243
24168104128.20977 2102068086.6951163
173122759.41069156 24168104128.20977
192969055.69907475 173122759.41069156
196591332.92605093 192969055.69907475
208658159.21535707 196591332.92605093
423861304.10695934 208658159.21535707
0.6535187433669711 423861304.10695934
0.0018833393180603952 0.6535187433669711
0.7326189947255094 0.0018833393180603952
0.0018833393180603952 0.732618

0.0024751844482224783 22.581107721141038
1141078467.7270277 0.0024751844482224783
3055067659.2575397 1141078467.7270277
40503262374.30271 3055067659.2575397
965791003.3546768 40503262374.30271
217628550.6465458 965791003.3546768
1039969554.2587192 217628550.6465458
1065658269.346272 1039969554.2587192
459222431.03357875 1065658269.346272
0.9380949058766255 459222431.03357875
0.0024751844482232555 0.9380949058766255
0.9479956436695187 0.0024751844482232555
0.0024751844482232555 0.9479956436695187
0.9479956436695187 0.0024751844482232555
0.0024751844482232555 0.9479956436695187
1509677 07
30157143370.715954 1509677.0
19.80461737585268 30157143370.715954
20.948065784949772 19.80461737585268
0.002273257274545415 20.948065784949772
695707169.7256458 0.002273257274545415
3209230386.1190224 695707169.7256458
31783920936.751343 3209230386.1190224
384256664.1935565 31783920936.751343
473003249.82365304 384256664.1935565
498143871.7816319 473003249.82365304
558380522.3404869 498143871.7816319
96

90656545.03751647 469728434.06998485
619330167.300593 90656545.03751647
707300140.8755583 619330167.300593
493297319.19138443 707300140.8755583
1.3442216778626748 493297319.19138443
0.002521991890924502 1.3442216778626748
1.6014648507369578 0.002521991890924502
0.002521991890924502 1.6014648507369578
1.7679163155379647 0.002521991890924502
0.00252199189092428 1.7679163155379647
1708476 07
32542857697.058712 1708476.0
21.661766861898844 32542857697.058712
23.32518766431049 21.661766861898844
0.0024864287031540755 23.32518766431049
551547383.0025675 0.0024864287031540755
4603618213.387717 551547383.0025675
35485177469.37894 4603618213.387717
246304941.8020425 35485177469.37894
479979574.53109854 246304941.8020425
351023355.38296264 479979574.53109854
398320136.0136266 351023355.38296264
1323051540.0105672 398320136.0136266
1.27553792471925 1323051540.0105672
0.002486428703156296 1.27553792471925
1.752932235725285 0.002486428703156296
0.002486428703156518 1.752932235725285
1.9568193893841

486488407.50859874 43349141790.578026
307385378.5474117 486488407.50859874
602614727.1875174 307385378.5474117
684541235.0549967 602614727.1875174
791915662.2346255 684541235.0549967
1.3733376405596736 791915662.2346255
0.0028551718098954737 1.3733376405596736
1.6417237906898383 0.0028551718098954737
0.0028551718098956957 1.6417237906898383
1.847296161002305 0.0028551718098956957
0.0028551718098954737 1.847296161002305
1886152 07
36585714908.765305 1886152.0
24.150655792707298 36585714908.765305
25.572750193724154 24.150655792707298
0.0027721138421377134 25.572750193724154
1079560301.7515278 0.0027721138421377134
3456022434.598495 1079560301.7515278
38902195329.66626 3456022434.598495
720155219.9110638 38902195329.66626
432578275.79310673 720155219.9110638
855973799.9832942 432578275.79310673
936133508.8543416 855973799.9832942
1067825171.7025323 936133508.8543416
1.0561753738546236 1067825171.7025323
0.0027721138421379354 1.0561753738546236
1.272400253541397 0.0027721138421379354
0.00

...

0.00514758418642125 1.2199774521817663
1.3332243042830272 0.00514758418642125
0.005147584186421028 1.3332243042830272
1.384700146147237 0.005147584186421028
0.005147584186421028 1.384700146147237
3133290 07
52028572314.61274 3133290.0
41.95225655534023 52028572314.61274
48.20753447836444 41.95225655534023
0.004815456445747657 48.20753447836444
2243430888.8199887 0.004815456445747657
7935292767.303604 2243430888.8199887
68775031149.66737 7935292767.303604
1513930338.728144 68775031149.66737
517066302.388704 1513930338.728144
1764381544.1836994 517066302.388704
1889202814.7311091 1764381544.1836994
2053879862.0355847 1889202814.7311091
2.12361629257404 2053879862.0355847
0.0048154564457463245 2.12361629257404
2.4221745922102995 0.0048154564457463245
0.0048154564457463245 2.4221745922102995
2.566638285582683 0.0048154564457463245
0.0048154564457463245 2.566638285582683
3132971 07
39928572108.550964 3132971.0
38.40853566133185 39928572108.550964
43.21842000551378 38.40853566133185
0.004408

284274703.771543 0.0037543837563873694
2342914036.1346235 284274703.771543
34184897808.55019 2342914036.1346235
217446872.14888957 34184897808.55019
141359861.14355698 217446872.14888957
247026497.09613097 141359861.14355698
263176863.02287048 247026497.09613097
516150962.21356714 263176863.02287048
1.5430517238736374 516150962.21356714
0.0037543837563835947 1.5430517238736374
1.760805981743883 0.0037543837563835947
0.0037543837563835947 1.760805981743883
1.8884550294609237 0.0037543837563835947
0.0037543837563835947 1.8884550294609237
3301602 07
18700000318.45911 3301602.0
31.910830577285473 18700000318.45911
40.83721879088104 31.910830577285473
0.0036628593408281063 40.83721879088104
968095427.0154134 0.0036628593408281063
3821582611.257207 968095427.0154134
34406933284.017876 3821582611.257207
607060309.7385179 34406933284.017876
338045238.8796066 607060309.7385179
803183786.0701317 338045238.8796066
889656267.5133363 803183786.0701317
1049127473.0085249 889656267.5133363
2.03288693

127884490.1611236 773660227.3987964
855153530.3152264 127884490.1611236
881729043.4631127 855153530.3152264
190962851.47974408 881729043.4631127
1.6344119045122203 190962851.47974408
0.0038822135499101407 1.6344119045122203
1.7741715923089902 0.0038822135499101407
0.0038822135499103627 1.7741715923089902
1.828522582007734 0.0038822135499103627
0.0038822135499103627 1.828522582007734
3521366 07
1133142876440.1655 3521366.0
137.81030184593033 1133142876440.1655
149.62668103313302 137.81030184593033
0.015818446033762257 149.62668103313302
103711856718.68958 0.015818446033762257
86825824428.4065 103711856718.68958
1260134648951.963 86825824428.4065
73593050894.47774 1260134648951.963
8406444258.961123 73593050894.47774
81523490362.28838 8406444258.961123
85309022283.61316 81523490362.28838
27766262076.80473 85309022283.61316
4.16025130687324 27766262076.80473
0.015818446033738276 4.16025130687324
4.982810500627645 0.015818446033738276
0.015818446033738276 4.982810500627645
5.29917942130241

534570555.4082481 466410922.8868675
1240441542.752729 534570555.4082481
1.995777148745987 1240441542.752729
0.004977000370937512 1.995777148745987
2.274489169518494 0.004977000370937512
0.004977000370937734 2.274489169518494
2.6826031999353797 0.004977000370937734
0.004977000370937734 2.6826031999353797
3841313 07
1191857163154.3496 3841313.0
158.3868240048515 1191857163154.3496
171.74934875732689 158.3868240048515
0.018180305785648443 171.74934875732689
126828838854.0728 0.018180305785648443
75025522539.43158 126828838854.0728
1318289981104.0132 75025522539.43158
92084524692.03189 1318289981104.0132
10104228276.716694 92084524692.03189
106730820265.67241 10104228276.716694
113753468964.37474 106730820265.67241
25214532142.522522 113753468964.37474
5.18138714891904 25214532142.522522
0.018180305785680417 5.18138714891904
6.599451000202144 0.018180305785680417
0.018180305785680417 6.599451000202144
7.290302620058018 0.018180305785680417
0.018180305785681306 7.290302620058018
3842859 07


75369068974.28714 8382335900.626587
654452255.990647 75369068974.28714
895198069.8317387 654452255.990647
739440185.2669827 895198069.8317387
880237214.6145191 739440185.2669827
3297226331.82862 880237214.6145191
2.7374085541333164 3297226331.82862
0.005691078075121148 2.7374085541333164
3.181312643992773 0.005691078075121148
0.005691078075121148 3.181312643992773
4.023592199110716 0.005691078075121148
0.005691078075121148 4.023592199110716
4139233 07
1242857164022.8745 4139233.0
179.88993310059703 1242857164022.8745
193.86898322790466 179.88993310059703
0.020648523083167447 193.86898322790466
138323459341.05615 0.020648523083167447
59496165816.465675 138323459341.05615
1357844644726.0117 59496165816.465675
105773919079.92825 1357844644726.0117
5527064241.47221 105773919079.92825
120732991217.96706 5527064241.47221
128395699032.50311 120732991217.96706
22718572520.52674 128395699032.50311
5.967423171036794 22718572520.52674
0.020648523083172776 5.967423171036794
7.371522740692509 0.020

In [82]:
import glob

VELA_number = '07'
properfiles = glob.glob('/Users/user1/documents/GEN3/rockstar_moresnaps/propermvircalc/VELA{}/improvedrvir[0-9][0-9][0-9].ascii'.format(VELA_number))

values_to_add = []

for properfile in properfiles:
                    
    readfile = open(properfile)
    lines = readfile.readlines()

    count = 0
    for line in lines:
        #we want to skip the first line as it is just the headers
        if count == 0:
            count +=1
            continue

        current_line = line.split()
        current_line_sql = [VELA_number] + [str(x) for x in current_line]
        values_to_add.append(tuple(current_line_sql))


    readfile.close()

print(values_to_add)

[('07', '581484.0', '16828571715.1603', '11.46821651745211', '12.251456741038426', '0.0013163701236749148', '262672564.91698104', '1969204215.4767535', '18280588176.089523', '107088391.15260203', '85428929.07826541', '127970524.14944322', '127970524.14944322', '142200086.43064845', '232180170.57518706', '0.546293601324911', '0.0013163701236744707', '0.6489704709715207', '0.0013163701236744707', '0.7147889771552449', '0.0013163701236744707'), ('07', '619344.0', '19914286053.42399', '12.359287021488582', '13.064356540506008', '0.00141865094369642', '321618879.6705799', '2255651405.377352', '20940273798.517582', '120586618.26247792', '109137185.28167681', '145716056.33226484', '145716056.33226484', '162340650.51217398', '290942878.11289334', '0.5461806133233591', '0.0014186509436971972', '0.6454861793821516', '0.0014186509436969752', '0.7164187265670032', '0.0014186509436969752'), ('07', '657580.0', '21114286073.859867', '12.818935754898373', '13.541398731901943', '0.0014714113584588517',

In [28]:
#need to add VELA_number

TypeError: 'NoneType' object is not iterable